In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path
import polars as pl
# import pandas as pd
import requests
from tqdm import tqdm

from src.paths import *
from src.logger import get_logger
from src.data import file_etl, batch_etl,  download_file_from_source_into_raw_folder, validate_file, aggregate_pickup_into_timeseries_data, generate_surrogate_key

logger = get_logger(__name__)

# Download files

In [3]:

# def download_file_from_source_into_raw_folder(year:int, month:int) -> Path:
#     file = RAW_DATA_DIR / Path(FILE_PATTERN.format(year=year, month=month))
#     url = BASE_URL.format(year=year, month=month)
#     response = requests.get(url)
#     response.raise_for_status()
#     if response.status_code == 200:
#         logger.info(f"Downloading file from {url} to {file}")
#         with open(file, "wb") as f:
#             f.write(response.content)
#     return file




# Validate a file
- Timestamp are within the file month. e.g, a file named `2020-07-01.csv` should have all timestamps within July 2020.

In [4]:
# df = pl.read_parquet(path)
# df.head()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
i64,datetime[ns],datetime[ns],f64,f64,f64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,null
1,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.2,1.0,"""N""",238,239,1,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5,null
1,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.2,1.0,"""N""",239,238,1,7.0,3.0,0.5,1.5,0.0,0.3,12.3,2.5,null
1,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.6,1.0,"""N""",238,238,1,6.0,3.0,0.5,1.0,0.0,0.3,10.8,2.5,null
1,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.8,1.0,"""N""",238,151,1,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0,null
2,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.0,1.0,"""N""",193,193,2,3.5,0.5,0.5,0.0,0.0,0.3,4.8,0.0,null


In [19]:
path = download_file_from_source_into_raw_folder(2020, 1)

df = (
    validate_file(path, 2020,1)
    .pipe(aggregate_pickup_into_timeseries_data, 2020,1)
    .pipe(generate_surrogate_key)
)



df.head()

key,pickup_datetime_hour,pickup_location_id,num_pickups
str,datetime[ns],i16,i16
"""2020-01-01 00:…",2020-01-01 00:00:00,1,42
"""2020-01-01 00:…",2020-01-01 00:00:00,2,0
"""2020-01-01 00:…",2020-01-01 00:00:00,3,6
"""2020-01-01 00:…",2020-01-01 00:00:00,4,653
"""2020-01-01 00:…",2020-01-01 00:00:00,5,1


In [20]:
file_etl(2020, 1)

Error deleting file C:\Users\selaf\OneDrive\Documentos\dev\real-ml-course\taxi_demand_predictor\data\raw\yellow_tripdata_2020-01.parquet: [WinError 32] El proceso no tiene acceso al archivo porque está siendo utilizado por otro proceso: 'C:\\Users\\selaf\\OneDrive\\Documentos\\dev\\real-ml-course\\taxi_demand_predictor\\data\\raw\\yellow_tripdata_2020-01.parquet'


# DuckDB

In [9]:
PARENT_DIR

WindowsPath('C:/Users/selaf/OneDrive/Documentos/dev/real-ml-course/taxi_demand_predictor')

In [31]:
# import duckdb


# DATABASE_URL = DATA_DIR / "dwh.duckdb"
# db = duckdb.connect(database=str(DATABASE_URL))


# def create_pickup_table(db: duckdb.DuckDBPyConnection):
#     db.execute("DROP TABLE IF EXISTS dwh.main.pickup_hourly;")
#     db.execute(
#         """
#         CREATE TABLE dwh.main.pickup_hourly (
#             key STRING PRIMARY KEY
#             , pickup_datetime_hour TIMESTAMP
#             , pickup_location_id SMALLINT
#             , num_pickup SMALLINT
#         );
#         """
#     )
    
    

    # return statement



In [3]:
file_etl(2020, 2)

In [6]:
batch_etl(year=2020)

100%|██████████| 12/12 [00:18<00:00,  1.58s/it]
